# Двоичное дерево
Binary Tree

Литература:
* Cormen et al, *Introduction to Algorithms*, Chapter 12

*Двоичное дерево* — структура данных, определённая на конечном множестве *узлов* (nodes), которая либо
* не содержит узлов,
* либо содержит узел (*корень*, root) и два других двоичных дерева, называемых соответственно левым и правым *поддеревьями* (subtree). Будем считать, что порядок поддеревьев фиксирован.

Корни поддеревьев (если они есть) называются соответственно левым и правым *потомками* (children) корня, связи между ними — *рёбрами*. Узлы без потомков называются *листовыми* (leaves), а остальные узлы — *внутренними* (internal).

*Степень* узла (degree) — количество потомков этого узла. Говорят, что узлы со степенью $i$ находятся на $i$-м уровне дерева.

*Глубина* узла (depth) – количество ребёр на пути от корня дерева этого узла.

*Высота* дерева — наибольшая глубина узлов в дереве.

Введём также несколько типов двоичных деревьев:

* *Полное* (full): все узлы имеют степень либо 2, либо 0.
* *Заполненное* (complete): полное дерево где у всех листьев одинаковая глубина.
* *Почти заполненное* (nearly complete): дерево, в котором заполненны все уровни за исключением последнего, который заполнен слева направо. Именно оно используется для двоичной кучи.

Для простых экспериментов с двоичными деревьями удобно использовать библиотеку [binarytree](https://github.com/joowani/binarytree):

In [1]:
from binarytree import tree, pprint, inspect

Случайное двоичное дерево:

In [2]:
T = tree(
    balanced=True
)
pprint(T)
inspect(T)


                    ____________________11___________________                     
                   /                                         \                    
         _________17_________                        _________4_________          
        /                    \                      /                   \         
     __8___               ____5___             ____16___             ____3___     
    /      \             /        \           /         \           /        \    
  _2       _29         _25        _7         26         _1        _30        _0   
 /  \     /   \       /   \      /  \       /  \       /  \      /   \      /  \  
14   9   12    19    13    22   18   27    6    15    28   20   21    10   24   23
                                                                                  


{'height': 4,
 'is_bst': False,
 'is_full': True,
 'is_height_balanced': True,
 'is_max_heap': False,
 'is_min_heap': False,
 'is_weight_balanced': True,
 'leaf_count': 16,
 'max_leaf_depth': 4,
 'max_value': 30,
 'min_leaf_depth': 4,
 'min_value': 0,
 'node_count': 31}

На практике (и в пакете `binarytree`) для представления узла двоичного дерева используется класс:

In [3]:
class Node(object):

    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None

In [4]:
from binarytree import Node

root = Node(10)
root.left = Node(6)
root.right = Node(11)
root.left.left = Node(1)
root.left.right = Node(5)

pprint(root)


    __10   
   /    \  
  6      11
 / \       
1   5      
           


Для некоторых алгоритмов также необходимо иметь ссылку на *родительский* узел (`self.parent`)

# Двоичное дерево поиска
Binary Search Tree

Пусть $x_v$ обозначает некоторое значение, связанное с узлом $x$.
*Двоичное дерево поиска* определяется следующим свойством:

<a id='bst_property'></a>
> Пусть $x$ — некоторый узел. Если $a$ — узел из левого поддерева $x$, то $a_v \le x_v$. Если $b$ — узел из правого поддерева $x$, то $b_v \ge x_v$. 

In [5]:
from binarytree import bst
pprint(bst())


         _18            
        /   \           
  _____15    21___      
 /                \     
6                 _23   
 \               /   \  
  9             22    26
   \                    
    14                  
                        


Согласно такому определению дерева поиска, если в дереве присутствуют узлы с одинаковыми значениями, то одно относительно другого может присутствовать как в левом, так и в правом поддереве:

In [6]:
from binarytree import convert

# Корректное (согласно определению) дерево поиска
pprint(convert([6, 5, 7, 2, 5, None, 8]))


    __6    
   /   \   
  5     7  
 / \     \ 
2   5     8
           


Наличие повторяющихся значений обычно усложняет алгоритмы, поэтому на практике в деревьях поиска часто хранят лишь уникальные значения.

In [23]:
# Вычисленное здесь дерево поиска
# будет использовано в примерах ниже

T = bst()
pprint(T)


  _____15____________      
 /                   \     
1              _______28   
 \            /         \  
  5         _22          30
   \       /   \           
    11    19    23         
                  \        
                   26      
                           


## Поиск элемента

[Свойство двоичного дерева поиска](#bst_property) позволяет использовать ту же стратегию, что и в поиске по отсортированному массиву:
* Если дерево пусто, сообщаем что элемент не найден.
* Сравниваем элемент со значением корнем дерева.
* Если значения совпадают — возвращаем найденный узел
* Если элемент больше, продолжаем поиск в правом поддереве.
* Если меньше — в левом поддереве.

Код рекурсивного алгоритма в точности повторяет это описание:

In [8]:
def search_recursive(node, k):
    if node is None: return
    
    if k == node.value:
        return node
    elif k > node.value:
        return search_recursive(node.right, k)
    else:
        return search_recursive(node.left, k)

In [25]:
search_recursive(T, 11)

Node(11)

Однако, пожертвовав выразительностью в пользу эффективности, перепишем его в итеративном виде:

In [10]:
def search(node, k):
    while node is not None and k != node.value:
        if k < node.value:
            node = node.left
        else:
            node = node.right
    return node

In [27]:
search(T, 15)

Node(15)

## Минимум и максимум

Из свойства дерева поиска также следует алгоритм поиска минимального и максимального значений: искать их надо соответственно в крайних левом и правом узлах:

In [12]:
def minimum(node):
    if node is None: return
    
    while node.left:
        node = node.left
    return node

def maximum(node):
    if node is None: return
    
    while node.right:
        node = node.right
    return node

In [28]:
print(repr(minimum(T)))
print(repr(maximum(T)))

Node(1)
Node(30)


## Обход дерева
Tree traversal

Под обходом дерева здесь подразумевается однократное применение некоторой функции `visit(node)` к каждому из узлов дерева. Например, эта функция может печатать, сравнивать или обновлять значения узлов. При этом порядок её применения к узлам зависит от используемого алгоритма.

### В глубину
Depth-first

При обходе дерева в глубину алгоритм, посетив узел $x$, переходит к другому узлу на той же глубине только после того, как посетит все узлы обоих поддеревьев с корнями $x_L$ и $x_R$.

Алгоритм можно записать рекурсивно. Функция `depth_first`, принимая на вход $x$, он выполняет три операции:
* Рекурсивно вызывает `depth_first` для поддерева с корнем $x_L$
* Рекурсивно вызывает `depth_first` для поддерева с корнем $x_R$
* Выполняет некоторую операцию `visit(x)`

В зависимости от того в каком порядке эти операции исполняются, обход в глубину делится на три типа:

In [14]:
def depth_first_preorder(node, visit):
    if node is None: return
    
    visit(node)
    depth_first_preorder(node.left, visit)
    depth_first_preorder(node.right, visit)

    
def depth_first_inorder(node, visit):
    if node is None: return
    
    depth_first_inorder(node.left, visit)
    visit(node)
    depth_first_inorder(node.right, visit)
    
    
def depth_first_postorder(node, visit):
    if node is None: return
    
    depth_first_postorder(node.left, visit)
    depth_first_postorder(node.right, visit)
    visit(node)

In [29]:
def visit(node):
    print(node.value, end=" ")

pprint(T)
print('Pre:')
depth_first_preorder(T, visit)
print()

print('In:')
depth_first_inorder(T, visit)
print()

print('Post:')
depth_first_preorder(T, visit)
print()


  _____15____________      
 /                   \     
1              _______28   
 \            /         \  
  5         _22          30
   \       /   \           
    11    19    23         
                  \        
                   26      
                           
Pre:
15 1 5 11 28 22 19 23 26 30 
In:
1 5 11 15 19 22 23 26 28 30 
Post:
15 1 5 11 28 22 19 23 26 30 


Обратите внимание, что `depth_first_inorder` обходит узлы в порядке их возрастания их значений (отсюда и название). Это можно использовать для проверки того, является ли двоичное дерево деревом поиска (т.е. выполяется ли его [свойство](#bst_property)).

### В ширину
Breadth-first

При обходе дерева в ширину алгоритм переходит к узлам с глубиной $n$ только после того как посетит все узлы глубины $n-1$. Иначе говоря, узлы будут обходиться «слева направо» начиная с корня и двигаясь по уровням вниз.

Проходя по узлам одного уровня, алгоритм «собирает» их потомков — узлы следующего уровня — в очередь, чтобы до них дошла очередь (pun intended) когда узлы текущего уровня исчерпаются.

In [16]:
from collections import deque

def breadth_first(node, visit):
    
    if node is None:
        return
    
    q = deque()
    q.append(node)
    
    while q:
        c = q.popleft()
        visit(c)
        if c.left: q.append(c.left)
        if c.right: q.append(c.right)

In [30]:
def visit(node):
    print(node.value, end=" ")

pprint(T)
breadth_first(T, visit)


  _____15____________      
 /                   \     
1              _______28   
 \            /         \  
  5         _22          30
   \       /   \           
    11    19    23         
                  \        
                   26      
                           
15 1 28 5 22 30 11 19 23 26 

## Следующий и предыдущий узел по порядку
Successor and Predecessor

Сначала воспользуемся обходом дерева чтобы добавить в узлы ссылку на «родителя»:

In [31]:
def visit(node):
    if node.left is not None:
        node.left.parent = node
        
    if node.right is not None:
        node.right.parent = node 

breadth_first(T, visit)

Свойство двоичного дерева поиска фактически поддерживает узлы в отсортированном порядке. Это даёт возможность для некоторого элемента найти в дереве элемент следующий за ним или предшествующий ему.

Рассмотрим поиск следующего по порядку элемента. Пусть дан узел $x$. Если у него существует поддерево $x_R$, то следующий элемент находится именно в нём — элементы из $x_R$ больше $x$, но меньше чем любые элементы уровнями выше $x$. Следующий за $x$ элемент будет минимумом из $x_R$.

Если же $x_R$ отсутствует, то элемент нужно искать уровнями выше. $x$ содержится в левом поддереве какого-то элемента, и является в нём максимальным (иначе следующий элемент мы бы выбирали из этого поддерева). Следовательно надо искать ближайшего «предка» $x$, для которого левый потомок также является предком $x$.

Поиск предыдущего элемента реализуется симметрично.

In [32]:
def successor(node):
    if node is None: return
    
    if node.right is not None:
        return minimum(node.right)
    
    else:
        ancestor = node.parent
        while (ancestor is not None) and (node is ancestor.right):
            node = ancestor
            ancestor = ancestor.parent
        
        return ancestor

def predecessor(node):
    if node is None: return
    
    if node.left is not None:
        return maximum(node.left)
    
    else:
        ancestor = node.parent
        while (ancestor is not None) and (node is ancestor.left):
            node = ancestor
            ancestor = ancestor.parent
        
        return ancestor

In [33]:
successor(search(T, 11))

Node(15)

# Другие рекурсивные алгоритмы

## Высота дерева

In [34]:
def height(node):
    if node is None:
        return -1
    return 1 + max(height(node.left), height(node.right))

In [35]:
print(height(T))
inspect(T)

4


{'height': 4,
 'is_bst': True,
 'is_full': False,
 'is_height_balanced': False,
 'is_max_heap': False,
 'is_min_heap': False,
 'is_weight_balanced': False,
 'leaf_count': 4,
 'max_leaf_depth': 4,
 'max_value': 30,
 'min_leaf_depth': 2,
 'min_value': 1,
 'node_count': 10}